In [42]:
import tensorflow as tf
import pandas as pd
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
from tokenizers import CharBPETokenizer
from pathlib import Path
import torch
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, BertTokenizerFast, GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import random
import time, math, copy
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchtext
from tokenizers import CharBPETokenizer
import tokenizers
import operator
from queue import PriorityQueue

# specify GPU
device = torch.device("cuda")

# Import from external file
X_train = pd.read_csv("output/x_train.csv")
X_test = pd.read_csv("output/x_test.csv")

# Set paths
train_path = 'output/x_train_nn.csv'
test_path = 'output/x_test_nn.csv'

# Save files
X_train[['question', 'prep_answer', 'cluster']].to_csv(train_path, index=False, header=False)
X_test[['question', 'prep_answer', 'cluster']].to_csv(test_path, index=False, header=False)

In [43]:
tokenizer = CharBPETokenizer(bert_normalizer=False)
# train the tokenizer model
tokenizer.train('output/x_train_nn.csv')
# saving the tokenized data in our specified folder
save_path = 'tokenized_data'
tokenizer.save_model(save_path)

['tokenized_data\\vocab.json', 'tokenized_data\\merges.txt']

In [44]:
from tokenise import BPE_token
from pathlib import Path
import os
#tokenizer = CharBPETokenizer(bert_normalizer=False)
# train the tokenizer model
#tokenizer.train('output/x_train_nn.csv')
# saving the tokenized data in our specified folder
#save_path = 'tokenized_data'
#tokenizer.save_model(save_path)
# the folder 'text' contains all the files
paths = [str(x) for x in Path("./output2/").glob("**/*.csv")]
tokenizer = BPE_token()
# train the tokenizer model
tokenizer.bpe_train(paths)
# saving the tokenized data in our specified folder
save_path = 'tokenized_data'
tokenizer.save_tokenizer(save_path)


In [45]:
# loading tokenizer from the saved model path
#tokenizer = GPT2Tokenizer.from_pretrained(save_path, return_tensors="tf")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})

# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
#model = TFGPT2LMHeadModel(config)
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


AttributeError: 'TFGPT2LMHeadModel' object has no attribute 'to'

In [36]:
single_string = ''
for i in range(len(X_train)):
    single_string += '<s> ' + 'Cluster' + str(X_train['cluster'].iloc[i]) + ' ' + X_train['prep_answer'].iloc[i] + ' </s> '
string_tokenized = tokenizer.encode(single_string)

In [37]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
#print(examples)
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
  #print(labels)

dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [38]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy()
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [39]:
num_epoch = 1
history = model.fit(dataset, epochs=num_epoch)

d:\programdata\venv\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


216/216 [==============================] - 1806s 8s/step - loss: 11.9120 - logits_loss: 11.9120 - logits_accuracy: 0.0047 - past_key_values_1_accuracy: 0.0018 - past_key_values_2_accuracy: 0.0025 - past_key_values_3_accuracy: 0.0021 - past_key_values_4_accuracy: 0.0018 - past_key_values_5_accuracy: 0.0015 - past_key_values_6_accuracy: 0.0030 - past_key_values_7_accuracy: 0.0019 - past_key_values_8_accuracy: 0.0036 - past_key_values_9_accuracy: 0.0013 - past_key_values_10_accuracy: 0.0016 - past_key_values_11_accuracy: 0.0016 - past_key_values_12_accuracy: 6.6298e-04

In [40]:
text = "<s> Cluster52"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  num_beams = 5,
  temperature = 0.7,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


In [41]:
print(tokenizer.decode(beam_output[0]))

Cluster52<s> you you person person and person you and and you facility person outbreak person facility and facility you outbreak and outbreak you advice and advice person advice you the person the andontario and the youontario personontario you, and effective and,
